<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/LEE/CodeBERT_1_N_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -U
!pip install transformers[torch] -U
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00


In [4]:
import pandas as pd

In [5]:
#df = pd.read_csv('/content/drive/MyDrive/Dataset/prototype5.csv')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model train/final_train1.csv')

<ipython-input-5-7cffaedba84c>:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/model train/final_train1.csv')


In [ ]:
df

In [ ]:
df.columns

In [6]:
df = df[['CWE ID','CODE', 'IDX']]

In [ ]:
df.columns

In [ ]:
df = df[df['CWE ID'] != 'CWE-119']

In [7]:
df = df[['CODE', 'IDX']]

In [8]:
import pandas as pd
import re
import transformers
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from pandas.core.common import random_state
import numpy as np
from datasets import load_metric
from transformers import TrainingArguments, Trainer
import logging
import sklearn.metrics as metric
from transformers import DataCollatorWithPadding
import torch,gc
from sklearn.metrics.pairwise import cosine_similarity
from torch.nn import CrossEntropyLoss
from transformers import AdamW
from transformers import RobertaForSequenceClassification, RobertaTokenizer, TrainingArguments, Trainer

In [9]:
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels = 11)
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
batch_size = 16
epoch_num = 7
MAX_LEN = 512
learning_rate = 2e-5

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [10]:
import re

def preprocess(df, file_name):
    # 멀티 라인 주석 제거
    preprocess_df = df.replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    preprocess_df = preprocess_df.replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include <.*?>\n'), "", regex=True)
    # double quotes를 사용하는 include 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#include \".*?\"\n'), "", regex=True)
    # 매크로 정의 제거
    preprocess_df = preprocess_df.replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    preprocess_df = preprocess_df.replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    preprocess_df = preprocess_df.replace(re.compile(r'\n\s*\n'), "\n", regex=True)
    preprocess_df = df.sample(frac=1).reset_index(drop=True)  # 랜덤 샘플링 후 인덱스 초기화

    preprocess_df.to_csv(file_name, index=False)

def tokenized(examples):
  return tokenizer(examples['CODE'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [11]:
preprocess(df,"preprocess.csv")
dataset = load_dataset("csv",data_files="preprocess.csv")['train']
encoded_dataset = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset=encoded_dataset.rename_column(original_column_name='IDX',new_column_name='labels')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/19164 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [33]:
encoded_dataset = encoded_dataset.train_test_split(0.3,seed=200)

In [34]:
optimizer = torch.optim.AdamW(model.parameters(),
            lr = learning_rate, betas=(0.9,0.99), eps=1e-8,
            weight_decay=0.1)
scheduler = transformers.get_cosine_schedule_with_warmup(optimizer,
                                        num_warmup_steps=5, num_training_steps=5)
optimizers = optimizer, scheduler

In [35]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # 각 클래스에 대한 F1 스코어 계산
    f1_per_class = f1_score(labels, predictions, average=None, labels=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

    # 가중 평균, 매크로 평균, 마이크로 평균 F1 스코어 계산
    f1_weighted = f1_score(labels, predictions, average='weighted')
    f1_macro = f1_score(labels, predictions, average='macro')
    f1_micro = f1_score(labels, predictions, average='micro')
    recall_weighted = recall_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # 결과를 딕셔너리 형태로 저장
    metrics = {
        'accuracy': accuracy,
        'recall_weighted': recall_weighted,
        'f1_weighted': f1_weighted,
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
    }

    # 클래스별 F1 스코어를 딕셔너리에 추가
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = f1

    return metrics


In [16]:
training_args = TrainingArguments(
    output_dir='./results',             # 학습 결과를 저장할 디렉터리
    num_train_epochs=7,                 # 데이터셋 크기에 맞춰 조정된 학습 에폭 수
    per_device_train_batch_size=16,     # 훈련 배치 크기, GPU 메모리에 맞게 조정 가능
    per_device_eval_batch_size=64,      # 평가 배치 크기, GPU 메모리에 맞게 조정 가능
    warmup_steps=500,                   # 워밍업 스텝 수, 총 스텝의 10%를 고려하여 조정 가능
    weight_decay=0.01,                  # 가중치 감소
    evaluation_strategy="epoch",        # 에폭마다 평가 수행
    logging_dir='./logs',               # 로그를 저장할 디렉터리
)

trainer = Trainer(
    model=model,                         # 학습시킬 모델
    args=training_args,                  # 학습 인자
    train_dataset=encoded_dataset['train'],         # 훈련 데이터셋
    eval_dataset=encoded_dataset['test'],           # 평가 데이터셋
    compute_metrics=compute_metrics,     # 메트릭 계산 함수
)


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [17]:
gc.collect()
torch.cuda.empty_cache()

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Recall Weighted,F1 Weighted,F1 Macro,F1 Micro,F1 Class 0,F1 Class 1,F1 Class 2,F1 Class 3,F1 Class 4,F1 Class 5,F1 Class 6,F1 Class 7,F1 Class 8,F1 Class 9,F1 Class 10
1,1.405700,0.650760,0.754957,0.754957,0.752668,0.742835,0.754957,0.866801,0.451939,0.521597,1.000000,0.950495,0.962679,0.969050,0.473389,0.850062,0.521930,0.603239
2,0.503200,0.423891,0.861913,0.861913,0.860527,0.853356,0.861913,0.932455,0.752381,0.672913,1.000000,0.984925,0.986916,0.977825,0.682422,0.887078,0.743624,0.766382
3,0.288100,0.373842,0.896870,0.896870,0.897008,0.894538,0.896870,0.922030,0.831008,0.758929,1.000000,0.984802,0.985130,0.983193,0.798552,0.912365,0.829508,0.834395
4,0.177200,0.373728,0.912522,0.912522,0.912535,0.911436,0.912522,0.943062,0.838977,0.783694,1.000000,0.987654,0.988084,0.984227,0.827160,0.928987,0.882083,0.861862
5,0.108300,0.407322,0.920000,0.920000,0.920043,0.915104,0.920000,0.953642,0.889580,0.811083,1.000000,0.980671,0.981481,0.980188,0.851169,0.927273,0.813423,0.877637
6,0.056800,0.398484,0.930261,0.930261,0.930682,0.927871,0.930261,0.955989,0.900763,0.843949,1.000000,0.980671,0.981481,0.986330,0.827508,0.939815,0.899851,0.890226
7,0.032300,0.416921,0.931130,0.931130,0.931360,0.929720,0.931130,0.956763,0.897959,0.830549,1.000000,0.980671,0.981481,0.983299,0.848000,0.940092,0.915515,0.892587


TrainOutput(global_step=5873, training_loss=0.3301534876798256, metrics={'train_runtime': 3252.3447, 'train_samples_per_second': 28.871, 'train_steps_per_second': 1.806, 'total_flos': 2.4707598270867456e+16, 'train_loss': 0.3301534876798256, 'epoch': 7.0})

In [19]:
trainer.save_model('model_final_train1')

In [20]:
import pandas as pd

df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/model train/final_val.csv")

In [21]:
df2 = df2[['CODE', 'IDX']]

In [22]:
df2

,CODE,IDX
0,nautilus_file_mark_desktop_file_trusted (GFile...,1
1,"int Track::Info::CopyStr(char* Info::*str, Inf...",1
2,long Tracks::ParseTrackEntry(long long track_s...,1
3,void HttpAuthFilterWhitelist::SetWhitelist(\n...,1
4,void CreateOAuth2ServiceDelegate(\n si...,1
...,...,...
475,static int au1200fb_ioctl(struct fb_info *info...,0
476,"static int nfs_read(struct device_d *dev, FILE...",0
477,static ChannelSecurityOptions *find_channel_se...,0
478,"uiserver_decrypt_verify (void *engine, gpgme_d...",0


In [23]:
import re

def preprocess2(df2, file_name):
    # 멀티 라인 주석 제거
    preprocess_df2 = df2.replace(re.compile(r'/\*.*?\*/', re.DOTALL), "", regex=True)
    # 싱글 라인 주석 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'//.*?\n'), "", regex=True)
    # angle brackets를 사용하는 include 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#include <.*?>\n'), "", regex=True)
    # double quotes를 사용하는 include 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#include \".*?\"\n'), "", regex=True)
    # 매크로 정의 제거
    preprocess_df2 = preprocess_df2.replace(re.compile(r'#define .*?\n'), "", regex=True)
    # 탭과 여러 공백을 하나의 공백으로
    preprocess_df2 = preprocess_df2.replace(re.compile(r'[\t ]+'), " ", regex=True)
    # 여러 줄바꿈을 하나로
    preprocess_df2 = preprocess_df2.replace(re.compile(r'\n\s*\n'), "\n", regex=True)
    preprocess_df2 = df2.sample(frac=1).reset_index(drop=True)  # 랜덤 샘플링 후 인덱스 초기화

    preprocess_df2.to_csv(file_name, index=False)

def tokenized(examples):
  return tokenizer(examples['CODE'], pad_to_max_length=True, max_length=MAX_LEN,truncation=True, return_token_type_ids=True)

In [29]:
preprocess2(df2,"preprocess2.csv")
dataset = load_dataset("csv",data_files="preprocess2.csv")['train']
encoded_dataset2 = dataset.map(tokenized,remove_columns=['CODE'],batched=True)
encoded_dataset2=encoded_dataset2.rename_column(original_column_name='IDX',new_column_name='labels')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/480 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2688: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [25]:
dataset

Dataset({
    features: ['CODE', 'IDX'],
    num_rows: 480
})

In [26]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13414
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5750
    })
})

In [37]:
trainer.predict(encoded_dataset['train'], metric_key_prefix="eval")

PredictionOutput(predictions=array([[-1.6745714 , -1.2094861 ,  0.7618732 , ..., -1.7733108 ,
        -0.600218  , -1.6481173 ],
       [-0.8746348 ,  0.14893766, -0.6602394 , ...,  9.48584   ,
        -1.3379364 , -1.4292421 ],
       [-2.0152214 , -0.46002835,  8.8226    , ..., -0.72414786,
         0.12271214, -1.3939756 ],
       ...,
       [10.248086  , -1.7749798 , -1.1594876 , ..., -0.77677137,
        -0.94967616, -1.4514884 ],
       [-0.7786235 , -0.81760174, -1.1790231 , ..., -0.8696156 ,
        -1.1289535 , -1.2134796 ],
       [-1.6366775 ,  0.32081598, -0.92814475, ..., -1.0871369 ,
        -0.13483082,  9.357434  ]], dtype=float32), label_ids=array([ 7,  8,  2, ...,  0,  5, 10]), metrics={'eval_loss': 0.02070990391075611, 'eval_accuracy': 0.9946324735351125, 'eval_recall_weighted': 0.9946324735351125, 'eval_f1_weighted': 0.9946348613513809, 'eval_f1_macro': 0.9943283438221386, 'eval_f1_micro': 0.9946324735351125, 'eval_f1_class_0': 0.9992855441771851, 'eval_f1_class_1'